# WASSA2023

## Dependencies

In [15]:
repo_path = "https://raw.githubusercontent.com/HLT-Ghisolfi-Leuzzi-Testa/WASSA-2023/"
branch = "main"

In [16]:
!pip install transformers -q
!pip install accelerate -U -q
!pip install datasets -q
!pip install torch-summary -q
!pip install graphviz -q
!pip install torchview -q
!pip install bertviz -q
!pip install NRCLex -q
!pip install textblob -q
!python -m textblob.download_corpora -q

In [17]:
utils_url = f"{repo_path}{branch}/utils.py"
evaluation_url = f"{repo_path}{branch}/evaluation.py"

import os
if os.path.exists("utils.py"):
  !rm "utils.py"
if os.path.exists("evaluation.py"):
  !rm "evaluation.py"

!wget {utils_url}
!wget {evaluation_url}

--2023-07-18 14:27:13--  https://raw.githubusercontent.com/HLT-Ghisolfi-Leuzzi-Testa/WASSA-2023/main/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38193 (37K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]  37.30K  --.-KB/s    in 0s      

2023-07-18 14:27:13 (90.2 MB/s) - ‘utils.py’ saved [38193/38193]

--2023-07-18 14:27:13--  https://raw.githubusercontent.com/HLT-Ghisolfi-Leuzzi-Testa/WASSA-2023/main/evaluation.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10675 (10K) [text/pl

In [18]:
import json
import torch
import string
from torch import nn
from torch.nn import BCEWithLogitsLoss
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel
from transformers import TrainingArguments, Trainer, TrainerCallback, EarlyStoppingCallback
from transformers import BertPreTrainedModel, BertModel
from transformers.modeling_outputs import SequenceClassifierOutput
import torch
from torch import nn
from transformers import AutoConfig, BertForSequenceClassification, RobertaForSequenceClassification, RobertaModel
from transformers.modeling_outputs import SequenceClassifierOutput
from typing import Optional, Union, Tuple
import importlib
import sys
from torch.utils.data import Dataset
from utils import *
importlib.reload(sys.modules['utils'])

<module 'utils' from '/content/utils.py'>

In [19]:
# set CUDA if available
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("======= CUDA Available =======")
else:
    device = torch.device('cpu')
    print("======= CUDA NOT Available, run on CPU =======")

======= CUDA Available =======


## Set up

In [20]:
EMOTIONS_NAMES = [
    'fear',
    'anger',
    'anticipation',
    'trust',
    'surprise',
    'positive',
    'negative',
    'sadness',
    'disgust',
    'joy',
    'hope'
]

config = {}

In [21]:
#@title Task
TASK = "EMP" #@param ["EMO", "EMP"]

In [22]:
#@title Number of labels
num_labels = 2 #@param {type:"integer"}
config['num_labels'] = num_labels

In [23]:
#@title Model Class
model_id = 'empathy_split0' #@param {type:"string"}
model_class_string = "RobertaPreTrainedModel" #@param ["BertPreTrainedModel", "RobertaPreTrainedModel"]
model_name = 'roberta-empathy' #@param ["bert-base", "roberta-base", "roberta-emotion", "distilroberta-emotion", "roberta-empathy"]

if model_class_string == "BertPreTrainedModel":
  model_class = BertForSequenceClassification
else:
  model_class = RobertaForSequenceClassification

config['model_id'] = model_id
config['model_class_string'] = model_class_string
config['model_name'] = model_name

In [24]:
#@title Model configuration
train_batch_size = 8 #@param {type:"integer"}
val_batch_size = 8 #@param {type:"integer"}
learning_rate = 5e-5 #@param {type:"number"}
weight_decay = 0.08 #@param {type:"number"}
epochs = 30 #@param {type:"integer"}
patience = 5 #@param {type:"integer"}
dropout = 0.3 #@param {type:"number"}
weighted_loss = "None" #@param ["None", "normalized_inverse", "balanced"]

model_type ={
  'distilroberta-emotion':'j-hartmann/emotion-english-distilroberta-base',
  'roberta-emotion':'j-hartmann/emotion-english-roberta-large',
  'roberta-empathy': 'bdotloh/roberta-base-empathy',
  'bert-base': 'bert-base-cased',
  'roberta-base' : 'roberta-base'
}

model_config = {
    'model_id': model_id,
    'tokenizer_name': model_type.get(model_name),
    'model_name': model_type.get(model_name),
    'train_batch_size': train_batch_size,
    'val_batch_size': val_batch_size,
    'learning_rate': learning_rate,
    'weight_decay': weight_decay,
    'epochs': epochs,
    'seed': 42,
    'patience': patience,
    'early_stopping_threshold': 0,
    'weighted_loss': weighted_loss,
    'dropout' : dropout
}



In [25]:
path_tosave = f"{TASK}_{model_config['model_id']}"
!mkdir $path_tosave

config.update(model_config)
write_dict_to_json(config, f"{path_tosave}/config.json")

print("\nCONFIGURATION")
for k,v in config.items():
  print(f"{k}: {v}")

mkdir: cannot create directory ‘EMP_empathy_split0’: File exists

CONFIGURATION
num_labels: 2
model_id: empathy_split0
model_class_string: RobertaPreTrainedModel
model_name: bdotloh/roberta-base-empathy
tokenizer_name: bdotloh/roberta-base-empathy
train_batch_size: 8
val_batch_size: 8
learning_rate: 5e-05
weight_decay: 0.08
epochs: 30
seed: 42
patience: 5
early_stopping_threshold: 0
weighted_loss: None
dropout: 0.3


## Dataset

### WASSA dataset

In [26]:
class WASSADataset(Dataset):

    def __init__(
        self,
        tokenizer,
        essay,
        targets,
        max_len=None
        ):
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.essay = essay
        self.targets = targets

    def __len__(self):
        return len(self.essay)

    def __getitem__(self, index):
        essay = str(self.essay[index])

        inputs = self.tokenizer.encode_plus(
            text=essay,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
            return_token_type_ids=True
        )

        item = {
          'input_ids': inputs['input_ids'].flatten(),
          'attention_mask': inputs['attention_mask'].flatten(),
          'token_type_ids': inputs["token_type_ids"].flatten()
        }

        if self.targets is not None:
          item['labels'] = torch.FloatTensor(self.targets[index])

        return item

Read dataframes

In [27]:
split = 0
TRAIN_DATA = f"{repo_path}{branch}/dataset_empathy/internal_train_split{split}.tsv"
VAL_DATA = f"{repo_path}{branch}/dataset_empathy/internal_val_split{split}.tsv"
DEV_DATA = f"{repo_path}{branch}/dataset_empathy/test_split{split}.tsv"
TEST_DATA = f"{repo_path}{branch}/datasets/WASSA23_essay_level_test_preproc.tsv"

train_df = pd.read_csv(TRAIN_DATA, sep='\t')
val_df = pd.read_csv(VAL_DATA, sep='\t')
dev_df = pd.read_csv(DEV_DATA, sep='\t')
test_df = pd.read_csv(TEST_DATA, sep='\t')

Encode targets

In [28]:
if TASK =="EMO":
  label_encoder = EmotionsLabelEncoder()
  label_encoder.fit(train_df.emotion)
  y_train = label_encoder.encode(train_df.emotion)
  y_val = label_encoder.encode(val_df.emotion)
  y_dev = label_encoder.encode(dev_df.emotion)

if TASK == "EMP":
  y_train = np.array(train_df[['empathy', 'distress']])
  y_val = np.array(val_df[['empathy', 'distress']])
  y_dev = np.array(dev_df[['empathy', 'distress']])

## Model

In [29]:
tokenizer = AutoTokenizer.from_pretrained(config.get('tokenizer_name'), truncation=True)
tokenizer

RobertaTokenizerFast(name_or_path='bdotloh/roberta-base-empathy', vocab_size=50265, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)}, clean_up_tokenization_spaces=True)

In [30]:
if TASK == "EMO":
  problem_type = "multi_label_classification"
if TASK == "EMP":
  problem_type = "regression"

model = AutoModelForSequenceClassification.from_pretrained(config.get('model_name'),
                                                          problem_type = problem_type,
                                                          num_labels = config.get('num_labels'),
                                                          classifier_dropout = config.get('dropout'),
                                                          ignore_mismatched_sizes=True)
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

## Training

Prepare datasets

In [31]:
train_set = WASSADataset(tokenizer=tokenizer,
                    essay=train_df.essay,
                    targets = y_train
                )

val_set = WASSADataset(tokenizer=tokenizer,
                    essay = val_df.essay,
                    targets = y_val
                )
dev_set = WASSADataset(tokenizer=tokenizer,
                    essay = dev_df.essay,
                    targets = y_dev
                )

Set up training

In [32]:
if TASK == "EMO":
  metric_for_val = "eval_macro_f1"
  compute_metrics_trainer = compute_EMO_metrics_trainer
if TASK == "EMP":
  metric_for_val = "eval_avg_pearson"
  compute_metrics_trainer = compute_EMP_metrics_trainer

In [33]:
train_arguments = TrainingArguments(
    output_dir=f"./{config.get('model_name')}",
    per_device_train_batch_size = config.get('train_batch_size'),
    per_device_eval_batch_size = config.get('val_batch_size'),
    num_train_epochs = config.get('epochs'),
    evaluation_strategy = "steps",
    save_strategy = "steps",
    logging_strategy = "steps",
    logging_steps = 150,
    eval_steps = 150,
    save_steps = 150,
    learning_rate=config.get('learning_rate'),
    weight_decay=config.get('weight_decay'),
    load_best_model_at_end=True,
    metric_for_best_model = metric_for_val,
    seed=config.get('seed'),
) # TODO: custom other params

In [34]:
trainer = Trainer(
    model=model,
    args=train_arguments,
    train_dataset=train_set,
    eval_dataset=val_set,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics_trainer
)

Callbacks

In [35]:
class TrainerLoggingCallback(TrainerCallback):
    def __init__(self, log_path):
        self.log_path = log_path

    def on_log(self, args, state, control, logs=None, **kwargs):
        _ = logs.pop("total_flos", None)
        if state.is_local_process_zero: # whether this process is the main one in a distributed setting
            with open(self.log_path, "a") as f:
                f.write(json.dumps(logs) + "\n")

trainer.add_callback(EarlyStoppingCallback(
    early_stopping_patience = config.get('patience'),
    early_stopping_threshold = config.get('early_stopping_threshold')))

trainer.add_callback(TrainerLoggingCallback(config.get('model_id')+"_log.json"))

Start training

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


In [ ]:
# Access the training logs
train_logs = trainer.state.log_history

metrics = [list(log.keys())[:-5] for log in train_logs if log.get('eval_loss') is not None][0]

train_loss_values = []
eval_loss_values = []
eval_metrics_values = []
for _ in metrics:
  eval_metrics_values.append([])
train_epochs = []
eval_epochs = []

best_metric = 0
best_train_loss = float('inf')
best_eval_loss = float('inf')

for log in train_logs:

  if log.get(metric_for_val) is not None:

    if log.get(metric_for_val) > best_metric:
      best_metric = log.get(metric_for_val)
      best_steps = log.get('step')
      best_epoch_metric = log.get('epoch')
    if log.get('eval_loss') < best_eval_loss:
      best_eval_loss = log.get('eval_loss')
      best_epoch_eval_loss = log.get('epoch')

    for i, metric in enumerate(metrics):
      eval_metrics_values[i].append(log.get(metric))

    eval_loss_values.append(log.get('eval_loss'))
    eval_epochs.append(log.get('epoch'))

  if log.get('loss') is not None:
    if log.get('loss') < best_train_loss:
      best_train_loss = log.get('loss')

    train_loss_values.append(log.get('loss'))
    train_epochs.append(log.get('epoch'))


Plot losses

In [ ]:
fig_name = "losses"
plot_metric_curve(
    values = [train_loss_values, eval_loss_values],
    epochs = [train_epochs, eval_epochs],
    metrics = ["train loss", "eval loss"],
    title = fig_name,
    path = f"{path_tosave}/{fig_name}.png")

Plot metrics

In [ ]:
fig_name = "metrics"
plot_metric_curve(
    values = eval_metrics_values[1:],
    epochs = [eval_epochs for _ in eval_metrics_values[1:]],
    metrics = metrics[1:],
    title = fig_name,
    path = f"{path_tosave}/{fig_name}.png")

## Evaluation

In [ ]:
def predict_emotions(results, gold_emotions):

  binarized_predictions = np.where(results >= 0.5, 1, 0)

  for i, bin_pred in enumerate(binarized_predictions):
    if np.all(bin_pred==0):
      binarized_predictions[i][np.argmax(results[i])] = 1

  predicted_emotions = label_encoder.decode(binarized_predictions)
  return predicted_emotions


In [ ]:
print(trainer.state.best_model_checkpoint)

outs = trainer.predict(dev_set)

In [ ]:
if TASK == "EMO":
  golds = label_encoder.decode(outs.label_ids)
  predictions = predict_emotions(outs.predictions, golds)
if TASK == "EMP":
  golds = outs.label_ids
  predictions = outs.predictions

In [ ]:
path_predictions = f"{path_tosave}/predictions_split{split}.tsv"
path_metrics = f"{path_tosave}/dev_metrics_split{split}.json"

scores = {
    'train_loss': float(best_train_loss),
    'eval_loss': float(best_eval_loss)
}

write_predictions(predictions, path_predictions)
challenge_metrics = compute_metrics(golds=golds, predictions=predictions, task=TASK)
scores.update(challenge_metrics)
scores['best_metric'] = float(best_metric)
scores['best_epoch_metric'] = float(best_epoch_metric)
scores['best_epoch_eval_loss'] =  float(best_epoch_eval_loss)

write_dict_to_json(scores, path_metrics)

scores

In [ ]:
if TASK == "EMO":
  fig_name = "confusion_matrix"
  plot_confusion_matrix(golds=golds,
                        predictions=predictions,
                        path= f"{path_tosave}/{fig_name}.png"
                        )
  print("\n")
  fig_name = "confusion_matrix_per_emotions"
  plot_confusion_matrix_per_emotions(gold_emotions = golds,
                                     predicted_emotions = predictions,
                                     path= f"{path_tosave}/{fig_name}.png"
                                     )

if TASK == "EMP":
  fig_name = f"empathy_true_vs_predicted"
  plot_true_vs_predicted(golds = golds[:,0],
                        predictions = predictions[:,0],
                        path= f"{path_tosave}/{fig_name}.png"
                        )
  print("\n")
  fig_name = f"distress_true_vs_predicted"
  plot_true_vs_predicted(golds = golds[:,1],
                        predictions = predictions[:,1],
                        path= f"{path_tosave}/{fig_name}.png"
                        )
  print("\n")
  fig_name = f"abs_diff_true_vs_predicted"
  plot_abs_diff_emp(golds = golds,
                        predictions = predictions,
                        path= f"{path_tosave}/{fig_name}.png"
                        )

# Predictions on Test

In [ ]:
test_set = WASSADataset(tokenizer=tokenizer,
                    essay = test_df.essay,
                    targets = None
                )

In [ ]:
# predict on test set
test_outs = trainer.predict(test_set)
test_predictions = test_outs.predictions
test_path_predictions = f"{path_tosave}/test_predictions_split{split}.tsv"
write_predictions(test_predictions, test_path_predictions)